In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt 



import pyspark
from pyspark.sql.types import *
from pyspark.ml.tuning import TrainValidationSplit
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator


# Build our Spark Session and Context
spark = pyspark.sql.SparkSession.builder.getOrCreate()
sc = spark.sparkContext
spark, sc


from pyspark.sql.functions import lit
from pyspark.sql.types import StructType, StructField, IntegerType
from pyspark.sql.functions import countDistinct, col

In [2]:
movie_data = pd.read_csv('data/movies.dat',
                                   sep="\t|::",
                                   names=['movie_id','title','genres'], 
                                   header=None, 
                                   engine="python")

In [5]:
def load_data_to_spark():
    df = pd.read_csv('data/training.csv')
    s_df = spark.createDataFrame(df)
    return s_df, df
 

In [6]:
s_df, p_df = load_data_to_spark()

In [16]:
s_df.show()

+----+-----+------+---------+
|user|movie|rating|timestamp|
+----+-----+------+---------+
|6040|  858|     4|956703932|
|6040|  593|     5|956703954|
|6040| 2384|     4|956703954|
|6040| 1961|     4|956703977|
|6040| 2019|     5|956703977|
|6040| 1419|     3|956704056|
|6040|  573|     4|956704056|
|6040| 3111|     5|956704056|
|6040|  213|     5|956704056|
|6040| 3505|     4|956704056|
|6040| 1734|     2|956704081|
|6040|  912|     5|956704191|
|6040|  919|     5|956704191|
|6040| 2503|     5|956704191|
|6040|  527|     5|956704219|
|6040|  318|     4|956704257|
|6040| 1252|     5|956704257|
|6040|  649|     5|956704257|
|6040| 3289|     5|956704305|
|6040|  759|     5|956704448|
+----+-----+------+---------+
only showing top 20 rows



In [17]:
def get_density():
    # get density from original data
    p_df, _ = load_data_to_spark()
    n_ratings = s_df.count()
    n_users = s_df.select('user').distinct().count()
    n_movies = s_df.select('movie').distinct().count()
    density = n_ratings / (n_users * n_movies)
    print('The original density is: {} '.format(density))
    return density

In [7]:
def traintestsplit():
    ratings_df, _ = load_data_to_spark()
    train, test = ratings_df.randomSplit([0.8, 0.2], seed=427471138)
    return train, test

In [19]:
def get_train_density():
    # get density from original data
    train, _ = traintestsplit()
    n_ratings = train.count()
    n_users = train.select('user').distinct().count()
    n_movies = train.select('movie').distinct().count()
    density = n_ratings / (n_users * n_movies)
    print('The train desnsity is: {} '.format(density))
    return density

In [20]:
def get_test_density():
    # get density from original data
    _, test = traintestsplit()
    n_ratings = test.count()
    n_users = test.select('user').distinct().count()
    n_movies = test.select('movie').distinct().count()
    density = n_ratings / (n_users * n_movies)
    print('The test desnsity is: {} '.format(density))
    return density

In [21]:
get_density()

The original density is: 0.04046302241176001 


0.04046302241176001

In [22]:
get_train_density()

The train desnsity is: 0.03257245318242673 


0.03257245318242673

In [23]:
get_test_density()

The test desnsity is: 0.008776990345157408 


0.008776990345157408

In [9]:
# instantiate the model and set its parameters
als_model = ALS(
    itemCol='movie',
    userCol='user',
    ratingCol='rating',
    nonnegative=True,    
    regParam=0.1,
    rank=10) 

In [26]:
# fitting
train, test = traintestsplit()
train_df = train.toPandas()
test_df = test.toPandas()
train_df.to_csv('./data/a_training.csv')
test_df.to_csv('./data/a_testing.csv')
recommender = als_model.fit(train)

In [36]:
# find if there are some movies that do not appear in train
# test_df.drop(cols = ['rating', 'timestamp'], axis = 1, inplace=True)
# test_df.to_csv('data/a_requests.csv')
train_df


,user,movie,rating,timestamp
0,3375,265,1,967595224
1,3375,293,3,967595382
2,3375,434,4,967595560
3,3375,1090,5,967595382
4,3375,1562,3,967595188
5,3375,1587,3,967595560
6,3375,1625,4,967595451
7,3375,2000,5,967595474
8,3375,2001,4,967595513
9,3375,2302,5,967595474


In [37]:
# distinct = s_df.select("movie").distinct().show()
train_pd = train.toPandas()
# distinct = p_df.movie.unique()
# distinct
distinct = train_pd.movie.unique()

In [38]:
not_in_train = []
for row in s_df.rdd.collect():
    movie = row['movie']
    if movie not in distinct:
        not_in_train.append(movie)
not_in_train
    
    

[3522,
 530,
 2685,
 1842,
 887,
 1709,
 3228,
 3337,
 1685,
 2685,
 1470,
 655,
 139,
 1316,
 989,
 2685,
 2217,
 2484,
 579,
 601,
 3220,
 1118,
 3377,
 641,
 3126,
 1471,
 2556]

In [39]:
# idea for cold start - if movie doesn't show up in test, don't recommend it in top 5
# item to item similarity
# neighborhoods - size of these that you use
# use combinations of different models 
# types of similarity - cosine similarity
# cold start - user shows up that hasn't rated any movies or movie thats never been reviewed
# 

In [11]:
# Light FM
predictions = recommender.transform(test)
predictions.show()

+----+-----+------+---------+----------+
|user|movie|rating|timestamp|prediction|
+----+-----+------+---------+----------+
|1242|  148|     3|974909976| 2.9531837|
|3539|  148|     3|966932408| 2.7834134|
|1150|  148|     2|974875106| 2.4366071|
|3829|  148|     2|965940170| 2.5057168|
|2507|  148|     4|974082717| 3.0551972|
|3841|  463|     3|966003085| 2.5498762|
|2629|  463|     4|973625620| 3.0239043|
|3328|  463|     4|967918151|  3.043021|
|3683|  463|     1|966523740| 1.4682665|
|3717|  463|     2|967228367| 2.7757328|
|3562|  463|     2|966790403| 2.6724248|
|5249|  463|     3|961602410|  2.762345|
|1699|  471|     5|974711905| 3.8341877|
|4167|  471|     5|965425364| 3.6452205|
|5337|  471|     4|960747681| 3.7248907|
|5880|  471|     5|957546755|  3.367366|
| 731|  471|     5|975529161|  3.800399|
|3211|  471|     4|968548060| 3.5678315|
|3411|  471|     2|967583402| 3.0830169|
|1019|  471|     4|975010672| 3.1851416|
+----+-----+------+---------+----------+
only showing top

In [12]:
predictions_pd = predictions.toPandas()

In [24]:
grouped = predictions_pd.groupby('user')
grouped2 = grouped.apply(lambda x: x.sort_values(['prediction']))
# df2= df1.apply(lambda x: x.sort_values(["lifeExp"]

In [25]:
grouped2

user  movie  rating  timestamp  prediction
user                                                   
636  96131    636   3695       3  975752411    1.829173
     108838   636   3693       3  975752146    3.033439
     65451    636   1971       5  975752317    3.200827
     25278    636   2668       3  975752288    3.245209
     149983   636   2456       2  975752349    3.254828
     159369   636   2746       5  975752146    3.489406
     17454    636    332       3  975752800    3.497720
     1087     636   2122       4  975752349    3.515913
     137055   636    379       3  975753026    3.574463
     102142   636   1320       4  975752996    3.616727
     43153    636   2699       4  975752740    3.759062
     117322   636   2615       4  975752963    3.768481
     16623    636   1372       3  975752834    3.788805
     100899   636   2012       5  975752908    3.968611
     155405   636   1097       4  975752543    4.245104
     93277    636   1261       5  975752226    4.264001
     131233   636    110       4  975751906    4.514980
     94814    636   1347       5  975752173    4.515113
637  111255   637   3564       3  975735656    1.837297
     56483    637   2950       1  975734497    2.181587
     153029   637    153       2  975734255    2.588854
     111203   637   2408       2  975734932    2.611277
     27724    637   2517       4  975734848    2.786248
     73949    637   2269       2  975736397    2.788669
     19483    637   3287       4  975739056    2.830111
     123260   637    801       3  975736096    2.920979
     51769    637   2613       4  975737438    3.047907
     94834    637   1347       4  975737466    3.065031
     11998    637   2003       4  975736010    3.070869
     157970   637   2021       2  975735337    3.073062
...           ...    ...     ...        ...         ...
6040 67831   6040   1231       2  956715421    3.553860
     89992   6040   1305       4  956715888    3.555764
     99173   6040   1214       4  960971715    3.563424
     48099   6040    920       3  957717358    3.599851
     141474  6040    778       4  960972645    3.676537
     42064   6040   1089       4  956704996    3.694507
     96349   6040   1094       5  956704887    3.706201
     102056  6040    759       5  956704448    3.707089
     76353   6040   2324       3  956704829    3.735895
     20871   6040   2076       5  956715676    3.736938
     17211   6040   3359       4  957717322    3.757111
     77817   6040   3201       4  957716904    3.761943
     73555   6040   1293       2  956715383    3.776984
     31047   6040   3504       4  960971857    3.822003
     37318   6040   1172       4  957717191    3.907267
     14209   6040   1296       2  956715383    3.932908
     28554   6040    246       5  956704613    3.940161
     9106    6040   1199       5  956715455    3.941306
     32471   6040   1206       4  957717631    3.959889
     15579   6040   1276       5  957716795    3.962036
     71986   6040   1219       5  957716861    3.998858
     89137   6040    649       5  956704257    4.010736
     80122   6040    800       4  960972032    4.026168
     29681   6040    111       5  957716717    4.031129
     93058   6040   1230       4  957716717    4.097969
     83212   6040    930       4  957717274    4.123058
     123016  6040    334       4  957717503    4.159794
     34545   6040    912       5  956704191    4.287793
     98860   6040    750       5  957716639    4.343410
     64895   6040   2019       5  956703977    4.355123

[160402 rows x 5 columns]

In [ ]:
# run.py, runs recommender.py, submit.py
